# PyVG: Data Science to predict Video Games sales
>Equipe: Alexis Terrasse, Henri-François Mole, Hsan Drissi, Stephane Lelievre
>
>Promo: DS_Oct21
---
## Scraping Vandal - Alexis

<img src='https://media.vandalimg.com/common/1200x800/2016422115313.jpg' width=500></img>

### Importation des bibliothèques

In [ ]:
from bs4 import BeautifulSoup # Importation de BeautifulSoup
import requests # Importation de import requests 
from time import sleep # Importation de sleep
import pandas as pd # Importation de pandas sous l'alias pd
import numpy as np # Importation de numpy sous l'alias np
from tqdm import tqdm # Importation de tqdm
import datetime # Importation de datetime
import os # Importation de os

### Définition de fonctions

In [ ]:
##################################################################
######### Fonction de récupération des données du jeu  ###########
##################################################################

def retrieve_data(sp, plat): 
        
    platf= '\''+plat.lower()+'\''
        
    g_name= sp.select('.linkversion')[0].text.strip() # On récupère le titre du jeu 
    
    s_sp= sp.select('#sombreado')[0]
    if s_sp.select('li[class=\"t12 fnormal\"]') != [] :   
        g_release= s_sp.select('li[class=\"t12 fnormal\"]')[0].text[22:]
    else : g_release = None
        
    g_analisis= np.nan
    for item in sp.select('.linktotal'):
        item_str= str(item)
        if platf in item_str :
            g_analisis= item.text
            
    g_comunidad=np.nan
    for item in sp.select('.notacomunidad'):
        if item.select('span[class=falsolink]') != []:
            if platf in item.select('span[class=falsolink]')[0].attrs['data-ruta'] :
                g_comunidad= item.text
    
    g_nvotes= 0    
    for item in sp.select('.votoscomunidad'):
        if item.select('span') != []:             
            platf= '\''+plat.lower()+'\''
            if platf in item.select('span')[0].attrs['data-ruta']:
                if len(item.text) > 6:
                    g_nvotes= item.text[:-6]

    if sp.select('.span8 div div div .mt05 b') == []: 
        g_name_en= g_name
    else : 
        g_name_en= sp.select('.span8 div div div .mt05 b')[0].text.strip() 

    return(g_name, g_release, g_analisis, g_comunidad, g_nvotes, g_name_en)


##################################################################
###### Fonction de parcours des jeux de la page active    ########
##################################################################

def scrap_page_title(url):
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response  = requests.get(url, headers = user_agent, timeout=60) # On requête le serveur
    soup = BeautifulSoup(response.text, 'html.parser') #On parse le résultat avec BeautifulSoup
    
    for link in soup.select('.t11 div'):

            plat= link.select('span')[0].text
            

            game_url= 'https://vandal.elespanol.com'+link.select('a')[0].attrs['href']
            user_agent = {'User-agent': 'Mozilla/5.0'}
            response  = requests.get(game_url, headers = user_agent, timeout=60) # On requête le serveur
            soup = BeautifulSoup(response.text, 'html.parser') #On parse le résultat avec BeautifulSoup
            if game_url in broken_link:
                continue
            else :
                platform.append(plat)  
                
                print(game_url)

                p_name, p_release, p_analisis, p_comunidad, p_votes, p_name_en= retrieve_data(soup, plat)

                gname.append(p_name)
                
                release.append(p_release)

                analisis.append(p_analisis)

                comunidad.append(p_comunidad)

                n_votes.append(p_votes)

                name_en.append(p_name_en)     

                print('Name:',p_name,' -')
                

##################################################################
######### Fonction de parcours des pages par années   ############
##################################################################
                
def vandal_scrap(start_y,end_y):
    
    dict_annee={2017: 77, 2018: 758, 2019: 860, 2020: 865, 2021: 812, 2022: 139}
    for annee in range(start_y,end_y):
        print('année:', annee)
        current_page= 0
        while current_page != -1 :        
            url= url_start+str(current_page)+'/'+str(annee)
            headermap = {"User-Agent": "Mac Firefox"};
            markup = requests.get(url, headers=headermap).text
            soup= BeautifulSoup(markup,"lxml")                   
            
            if annee >= 2017 :
                if current_page+100 > dict_annee[annee]:
                    current_page = -1
                else: current_page+= 100
            else : 
                if (soup.select('.pildora')[0].text == 'Anteriores 100 puestos') and (len(soup.select('.pildora')) == 1):
                    current_page = -1
                else : current_page+= 100
            
            scrap_page_title(url)
        
        columns = {
            'ES_Name': gname,
            'Name': name_en,
            'Release_date': release,
            'platform': platform,
            'Analyst_Score': analisis,
            'Community_score': comunidad,
            'N_com_votes': n_votes,
        }
    
        df = pd.DataFrame(columns)

        file_name='vandal_'+str(annee)+'.csv'

        df.to_csv(file_name, sep=",", encoding='utf-8', index=False)

In [ ]:
def scrap_page_title(url):
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response  = requests.get(url, headers = user_agent, timeout=60) # On requête le serveur
    soup = BeautifulSoup(response.text, 'html.parser') #On parse le résultat avec BeautifulSoup
    
    for link in soup.select('.t11 div'):

            plat= link.select('span')[0].text
            

            game_url= 'https://vandal.elespanol.com'+link.select('a')[0].attrs['href']
            user_agent = {'User-agent': 'Mozilla/5.0'}
            response  = requests.get(game_url, headers = user_agent, timeout=60) # On requête le serveur
            soup = BeautifulSoup(response.text, 'html.parser') #On parse le résultat avec BeautifulSoup
            if game_url in broken_link:
                continue
            else :
                platform.append(plat)  
                
                print(game_url)

                p_name, p_release, p_analisis, p_comunidad, p_votes, p_name_en= retrieve_data(soup, plat)

                gname.append(p_name)
                
                release.append(p_release)

                analisis.append(p_analisis)

                comunidad.append(p_comunidad)

                n_votes.append(p_votes)

                name_en.append(p_name_en)     

                print('Name:',p_name,' -')


In [ ]:
# début de l'url du site trié par metascore et par année
url_start = 'https://vandal.elespanol.com/rankings/videojuegos/todos/vandal/' # début de l'url du site trié par metascore et par année

# Lien hs
broken_link = ['https://vandal.elespanol.com/juegos/pc/7th-sector/70512#p-83', 
               'https://vandal.elespanol.com/juegos/pc/7th-sector/70512#p-79',
               'https://vandal.elespanol.com/juegos/pc/7th-sector/70512#p-73',
               'https://vandal.elespanol.com/juegos/pc/7th-sector/70512#p-13',
              ]

    # Fonction (main) du scraping de metacritic

gname = []
release= []
platform =[]
analisis = []
comunidad = []
n_votes= []
name_en = []

In [ ]:
currentDateTime = datetime.datetime.now()
date = currentDateTime.date()
current_year = date.strftime("%Y")

In [ ]:
# Scraping des données de 2003 à année actuelle (si le csv n'existe pas déjà)
for i in range(2003, int(current_year)+1):
    vandal_year= './vandal_'+str(i)+'.csv'
    if os.path.exists(vandal_year) == False :
        vandal_scrap(i,i+1)

In [ ]:
# Regroupement des différents csv
vandal= pd.DataFrame()
for i in range(2003,2023): # Pas de données avant 1995
    filename= 'vandal_'+str(i)+'.csv'
    df= pd.read_csv(filename)
    vandal= pd.concat([vandal,df],ignore_index=True)

vandal= vandal.drop_duplicates(keep='first')
vandal.to_csv('vandal.csv', sep=",", encoding='utf-8', index=False)
vandal.head()